# Claim-based Question-Answer Hallucination Detection

Import necessary packages.

In [1]:
import time
from uqlm.longform.black_box import ClaimQAScorer
from uqlm import BlackBoxUQ
from uqlm.utils.prompt_templates import get_claim_breakdown_template

#### Load LLM and device

In [2]:
# from langchain_google_vertexai import ChatVertexAI
# llm = ChatVertexAI(model="gemini-1.5-flash")

from dotenv import load_dotenv, find_dotenv
from langchain_openai import AzureChatOpenAI

load_dotenv(find_dotenv())
llm = AzureChatOpenAI(
    deployment_name="gpt-4o-mini",
    openai_api_type="azure",
    openai_api_version="2024-02-15-preview",
    temperature=1,  # User to set temperature
)

In [3]:
import torch

# Set the torch device
if torch.cuda.is_available():  # NVIDIA GPU
    device = torch.device("cuda")
elif torch.backends.mps.is_available():  # macOS
    device = torch.device("mps")
else:
    device = torch.device("cpu")  # CPU
print(f"Using {device.type} device")

Using mps device


#### Setup Prompts and Black Box Scorer

In [4]:
prompts = ["write a paragraph about Paul McCartney", 
           # "write a paragraph about John Lennon"
           ]

In [5]:
bb_scorer = BlackBoxUQ(
    llm=llm,
    max_calls_per_min=500,  # set value to avoid rate limit error
    device=device,
    scorers = ["semantic_negentropy", "exact_match"]
)

#### Claim-QA class

In [6]:
start_time = time.time()
claim_qa = ClaimQAScorer(llm=llm, bb_scorer=bb_scorer, response_template=get_claim_breakdown_template)
result = await claim_qa.generate_and_score(prompts=prompts)
print(f"Computation time: {time.time() - start_time} seconds")

['Paul McCartney is an iconic British musician, singer, and songwriter, best known as a co-founder of the legendary rock band The Beatles. Born on June 18, 1942, in Liverpool, England, McCartney\'s melodic prowess and innovative songwriting played a pivotal role in the band\'s unprecedented success during the 1960s. His contributions to timeless hits like "Hey Jude," "Yesterday," and "Let It Be" remain integral to music history. After The Beatles disbanded in 1970, McCartney embarked on a successful solo career and formed the band Wings, further showcasing his versatility and staying power in the music industry. Beyond his musical achievements, McCartney is also known for his activism in animal rights and environmental issues, reflecting his deep commitment to social causes. His enduring influence and creativity continue to inspire generations of musicians and fans around the world.']
Claim sets:  [['Paul McCartney is an iconic British musician.  \n', 'Paul McCartney is a singer.  \n',

#### Decomposition

In [7]:
from uqlm.longform import ResponseDecomposer

In [8]:
rd = ResponseDecomposer(llm)

In [9]:
responses = [
    "Hello there! Today I visited Dr. Chauhan at his home. He was hanging his framed B.S., M.S., and Ph.D. degrees. It was fun!",
    "One time I went on a class trip to D.C. to visit the white house. While there, I felt in awe of the beutiful art and important people with titles such as Prof., Esq., and others."
]

sampled_responses = [
    [
        "Hi! I went to see Dr. Chauhan at his house today. He was busy putting up his framed B.S., M.S., and Ph.D. diplomas. It was quite enjoyable!",
        "Greetings! Today, I stopped by Dr. Chauhan's home. He was in the process of displaying his framed degrees: B.S., M.S., and Ph.D. It was a delightful experience!"
    ],
    [
        "Once, I took a school trip to Washington, D.C. to see the White House. While I was there, I was amazed by the stunning artwork and the notable individuals with titles like Professor, Esquire, and more.",
        "During a class excursion to Washington, D.C., I had the opportunity to visit the White House. I was struck by the beautiful art and the distinguished people holding titles such as Prof., Esq., and others."
    ]
    
]

In [10]:
# Decompose into sentences
rd.decompose_sentences(responses=responses)

[['Hello there!',
  'Today I visited Dr. Chauhan at his home.',
  'He was hanging his framed B.S., M.S., and Ph.D. degrees.',
  'It was fun!'],
 ['One time I went on a class trip to D.C. to visit the white house.',
  'While there, I felt in awe of the beutiful art and important people with titles such as Prof., Esq., and others.']]

In [11]:
# Decompose into claims
from uqlm.utils.prompt_templates import get_factoid_template
await rd.decompose_claims(responses=responses, response_template=get_factoid_template)

Claim sets:  [['The speaker visited Dr. Chauhan at his home today.  \n', 'Dr. Chauhan was hanging his framed B.S., M.S., and Ph.D. degrees.  \n', 'The visit was described as fun.  '], ['I went on a class trip to D.C. to visit the White House.  \n', 'While there, I felt in awe of the beautiful art.  \n', 'I felt in awe of important people with titles such as Prof., Esq., and others.']]


[['The speaker visited Dr. Chauhan at his home today.  \n',
  'Dr. Chauhan was hanging his framed B.S., M.S., and Ph.D. degrees.  \n',
  'The visit was described as fun.  '],
 ['I went on a class trip to D.C. to visit the White House.  \n',
  'While there, I felt in awe of the beautiful art.  \n',
  'I felt in awe of important people with titles such as Prof., Esq., and others.']]

In [12]:
from uqlm.utils.prompt_templates import get_claim_breakdown_template
await rd.decompose_claims(responses=responses, response_template=get_claim_breakdown_template)

Claim sets:  [['Today, I visited Dr. Chauhan at his home.  \n', 'I visited Dr. Chauhan.  \n', 'Dr. Chauhan is at his home.  \n', 'Dr. Chauhan was hanging his framed degrees.  \n', 'Dr. Chauhan was hanging his framed B.S. degree.  \n', 'Dr. Chauhan was hanging his framed M.S. degree.  \n', 'Dr. Chauhan was hanging his framed Ph.D. degree.  \n', 'It was fun.  '], ['I went on a class trip to D.C.  \n', 'I went to visit the White House.  \n', 'I felt in awe of the beautiful art.  \n', 'I felt in awe of important people.  \n', 'Important people have titles such as Prof., Esq., and others.  ']]


[['Today, I visited Dr. Chauhan at his home.  \n',
  'I visited Dr. Chauhan.  \n',
  'Dr. Chauhan is at his home.  \n',
  'Dr. Chauhan was hanging his framed degrees.  \n',
  'Dr. Chauhan was hanging his framed B.S. degree.  \n',
  'Dr. Chauhan was hanging his framed M.S. degree.  \n',
  'Dr. Chauhan was hanging his framed Ph.D. degree.  \n',
  'It was fun.  '],
 ['I went on a class trip to D.C.  \n',
  'I went to visit the White House.  \n',
  'I felt in awe of the beautiful art.  \n',
  'I felt in awe of important people.  \n',
  'Important people have titles such as Prof., Esq., and others.  ']]